In [1]:
#import sys
#!{sys.executable} -m pip install proxy-checker

In [2]:
# Load dependencies
import warnings
warnings.filterwarnings('ignore') #silence the depreciation warnings 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import requests 
import urllib.request as req
import time 
import random
import bs4 as bs #beautiful soup for web scraping 
from IPython.display import clear_output

In [3]:
#user agent from https://www.whatismybrowser.com/detect/what-is-my-user-agent
url = "https://www.zoocasa.com/toronto-on-sold-listings?page=1" #sold listings 
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
              #(KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36 Edg/92.0.902.84' 
headers = {'User-Agent':user_agent} 
#request = req.Request(url,None,kwargs) #assemble the request
#data_raw = req.urlopen(request).read() #read the request to html
data_raw = requests.get(url,headers=headers).text
data_raw

'<!DOCTYPE html><html lang="en"><head><script src="https://www.googleoptimize.com/optimize.js?id=OPT-KVMRCFH"></script><meta name="zoocasa-ember/config/environment" content="%7B%22podModulePrefix%22%3A%22zoocasa-ember%2Fpods%22%2C%22modulePrefix%22%3A%22zoocasa-ember%22%2C%22environment%22%3A%22production%22%2C%22rootURL%22%3A%22%2F%22%2C%22locationType%22%3A%22auto%22%2C%22EmberENV%22%3A%7B%22FEATURES%22%3A%7B%7D%2C%22EXTEND_PROTOTYPES%22%3A%7B%22Date%22%3Afalse%7D%2C%22_APPLICATION_TEMPLATE_WRAPPER%22%3Afalse%2C%22_DEFAULT_ASYNC_OBSERVERS%22%3Atrue%2C%22_JQUERY_INTEGRATION%22%3Afalse%2C%22_TEMPLATE_ONLY_GLIMMER_COMPONENTS%22%3Atrue%7D%2C%22APP%22%3A%7B%22host%22%3A%22https%3A%2F%2Fwww.zoocasa.com%22%2C%22analytics%22%3A%7B%22enabled%22%3Afalse%7D%2C%22features%22%3A%7B%22soldData%22%3Atrue%2C%22travelTime%22%3Atrue%2C%22playground%22%3Afalse%2C%22drawOnMap%22%3Afalse%2C%22schoolsListPage%22%3Afalse%7D%2C%22map%22%3A%7B%22provider%22%3A%22apple%22%7D%2C%22name%22%3A%22zoocasa-ember%22

#split the data 
data_split = data_raw.split('status')
data_split = data_split[1:] #the first list element is not necessary

data_split[2]

links = [];

for post in range(len(data_split)):
    start = data_split[post].find('addressPath')
    if start > 0:
        end = len(data_split[post])
        links.append(data_split[post][(start+14):(end-3)])

links

In [4]:
#find the total number of pages to loop through
start = data_raw.find('totalPages')
end = data_raw.find('totalCount')
total_pages = int(data_raw[(start+12):(end-2)])
total_pages

1453

In [5]:
#define a funcrtion to generate proxies to cycle through

from lxml.html import fromstring
import requests
from itertools import cycle
import traceback
from proxy_checker import ProxyChecker

def get_proxy():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxy = ''
    for i in parser.xpath('//tbody/tr')[:20]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            test_proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            checker = ProxyChecker()
            checkProxy = checker.check_proxy(test_proxy)
            if checkProxy != False:
                #this ensures that the proxy works 
                proxy = test_proxy
                return proxy
    #if nothing is returned, call the function again
    get_proxy() 

In [6]:
#define function to loop through set of pages and get links 

def get_links(pages):
    links = pd.DataFrame(columns=['link'])
    links_done = 0
    pages_skipped = []

    #list of user agents from https://www.whatismybrowser.com
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246' 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36' 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9' 
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1' 
    'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36' 
    ]


    for page in pages:
        clear_output(wait=True) #this clears the progress bar 

        url = f"https://www.zoocasa.com/toronto-on-sold-listings?page={page}"

        print(f'Pages Skipped:{pages_skipped}')
        print(f'Number of Duplicates: {len(links[links.duplicated(["link"])])}')
        print(f'Fetching Random Proxy For Page {page}...')

        #get random proxies and user-agent
        proxy = get_proxy()
        proxies = {"http": proxy, "https": proxy}
        user_agent = user_agent_list[random.randint(0,len(user_agent_list)-1)] #select random UA
        headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image\
                    apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 
                    'Accept-Encoding': 'gzip, deflate', 
                    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8', 
                    'User-Agent':user_agent 
                    } 
        payload = {
                    'inUserName': 'USERNAME/gianfelicemichael5@gmail.com', 
                    'inUserPass': 'Zoocasa123!'
                  }

        print(f'Connecting to Page {page}')
        print(url)

        #keep trying to connect 
        while True: 
            try:
                session = requests.Session()
                initial_login = session.get('https://www.zoocasa.com/',headers=headers,proxies=proxies).text
                #r = session.post(url, data=payload, headers=headers, proxies=proxies)
                time.sleep(random.randint(3,8))
                data_raw = ''
                data_raw = session.get(url,headers=headers,proxies=proxies).text
                s=data_raw.find('<title>')+7
                e=data_raw.find('</title>')
                isForbidden = data_raw[s:e].strip()=='403 Forbidden'
                if isForbidden:
                    #break the loop and skip page if access if forbidden
                    print(f'Access Forbidden! Page {page} Skipped')
                    pages_skipped.append(page)
                    break
                else:
                    data_split = []
                    data_split = data_raw.split('status')[1:] 
                    for post in range(len(data_split)):
                        start = data_split[post].find('addressPath')
                        if start > 0:
                            #start will be -1 for posts that do not have addressPath (i.e no housing data)
                            end = len(data_split[post])
                            link_name = data_split[post][(start+14):(end-3)]
                            if links['link'].str.contains(link_name).any():
                                print(f'Duplicate Link Detected, {page} Skipped')
                                pages_skipped.append(page)
                                break
                            else:
                                links.loc[links_done] = link_name
                                links_done += 1
                    session.close()
                    break 

            except Exception as e:
                print(e)
                print(f'Skipped Page {page}')
                pages_skipped.append(page)
                break 

        print(f'Progress:{np.round(page/(len(pages)-1)*100,2)}%')
        time.sleep(random.randint(15,25)) #set a random crawl delay

    print(f'Number of Duplicates: {len(links[links.duplicated(["link"])])}')
    print(f'Pages Skipped: {pages_skipped}')
    return links, pages_skipped

In [ ]:
#loop to go through all the pages on zoocasa and grab links
up_to_page = total_pages
pages = range(1,up_to_page+1)

keep_going = True 
iter_num = 1
while keep_going == True:
    links,pages_skipped = get_links(pages)
    links.to_csv('house_links_' + str(iter_num) + '.csv')
    pages = pages_skipped
    iter_num += 1
    if len(pages_skipped) == 0:
        keep_going == False 
        break
print('Done')

Pages Skipped:[2, 3, 4, 5, 6]
Number of Duplicates: 0
Fetching Random Proxy For Page 7...
Connecting to Page 7
https://www.zoocasa.com/toronto-on-sold-listings?page=7
